In [15]:
import pandas as pd

df = pd.read_csv("/kaggle/input/email-spam-detection-dataset-classification/spam.csv",encoding = "latin1")
df.head(10)


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [11]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline

# Load dataset (replace with your dataset path)
# For example, SMS Spam Collection dataset
df = df[['v1', 'v2']]
df.columns = ['label', 'text']

# Convert labels to numeric
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label_num'], test_size=0.2, random_state=42)

# Create a pipeline: TF-IDF Vectorizer + Naive Bayes
nb_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('nb', MultinomialNB())
])

# Train Naive Bayes
nb_pipeline.fit(X_train, y_train)

# Predict probabilities
nb_probs = nb_pipeline.predict_proba(X_test)

# Use predicted probabilities as features for a Decision Tree
# Optional: you can also add TF-IDF features here
dt_features = pd.DataFrame(nb_probs, columns=['prob_ham', 'prob_spam'])

# Train Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(dt_features, y_test)

# Predict using Decision Tree
dt_preds = dt.predict(dt_features)

# Evaluation
print("Accuracy:", accuracy_score(y_test, dt_preds))
print("Confusion Matrix:\n", confusion_matrix(y_test, dt_preds))
print("Classification Report:\n", classification_report(y_test, dt_preds))


Accuracy: 0.9982062780269059
Confusion Matrix:
 [[965   0]
 [  2 148]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       965
           1       1.00      0.99      0.99       150

    accuracy                           1.00      1115
   macro avg       1.00      0.99      1.00      1115
weighted avg       1.00      1.00      1.00      1115



In [ ]:
# Get probabilities from Naive Bayes for Decision Tree
nb_probs = nb_pipeline.predict_proba(X_test)
dt_features = pd.DataFrame(nb_probs, columns=['prob_ham', 'prob_spam'])

# Train Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(dt_features, y_test)

# Function to predict spam/ham in 0/1 format
def predict_spam_numeric(messages):
    probs = nb_pipeline.predict_proba(messages)
    probs_df = pd.DataFrame(probs, columns=['prob_ham', 'prob_spam'])
    preds = dt.predict(probs_df)
    return preds  # 0 = ham, 1 = spam

# Example: Predict numeric spam/ham for messages in the CSV
df['prediction'] = predict_spam_numeric(df['text'])

# Save results to a new CSV
df.to_csv('spam_predictions_numeric.csv', index=False)

# Preview results
print(df[['text', 'label_num', 'prediction']].head())

                                                text  label_num  prediction
0  Go until jurong point, crazy.. Available only ...          0           0
1                      Ok lar... Joking wif u oni...          0           0
2  Free entry in 2 a wkly comp to win FA Cup fina...          1           1
3  U dun say so early hor... U c already then say...          0           0
4  Nah I don't think he goes to usf, he lives aro...          0           0
